In [21]:
import pandas as pd
import numpy as np
import re

In [22]:
%%bigquery raw_exclusions
SELECT *
FROM `yhcr-prd-phm-bia-core.CY_FDM_DepartmentForEducation.src_exclusions`

Downloading: 100%|██████████| 45778/45778 [00:02<00:00, 20558.35rows/s]


# Exclusions table Description:

The immediate issue with the raw exclusions table is it's huge witdth. The table has 159 columns comprised of:

* 9 unique columns: person_id, EDRN, SourceTable, AcademicYear, PupilMatchingRefAnonymous, NumberOfEnrolments, TotalFixedExclusions, TotalFixedSessions, PermanentExclusionCount,  
* 5 columns that are repeated 30 times: Category, Reason, Term, StartDate, Sessions 

Each row represents one academic year for one student. The unique columns contain identifiers (person_id, EDRN, AcademicYear), aggregate statistics for the exclusions in that year (NumberOfEnrolments, TotalFixedExclusions, TotalFixedSessions, PermanentExclusionCount) and metadata (SourceTable, PupilMatchingRefAnonymous). The repeated columns contain the details of one exclusion incidence - they are repeated 30 times to accommodate the maximum number of exclusion incidences in one academic year, thus the vast majority of the repeated columns contain NULLs

To make the table more user-friendly, the number of columns needs reduction: 

To start, we can drop the EDRN and PupilMatchingRefAnonymous, as they contain no useful information that isn't already coded by the person_id. I don't think the source table really adds any information to the dataset, so that can also be dropped.

Then, we can split each of the repeated columns into it's own entry. Each is unique given they are all dated on the day the exclusion occurs, so there shouldn't be any issue with overlap that isn't a duplicated entry (as how a student be excluded more than once on the same date):

In [23]:
excl_cleaned = pd.DataFrame([])

for i in range(1, 31):
    non_num_cols = [ 
        'person_id', 'AcademicYear', 'NumberOfEnrolments', 
        'TotalFixedExclusions', 'TotalFixedSessions', 
        'PermanentExclusionCount'
    ]
    num_cols = [col for col in raw_exclusions.columns
                if re.sub('[^0-9]','', col) == str(i)]
    df = raw_exclusions[non_num_cols + num_cols]
    non_na_rows = ~df[num_cols].isna().all(axis=1)
    df = df[non_na_rows]
    rename_dict = {col: re.sub('[0-9]', '', col) 
                   for col in num_cols}
    df.rename(rename_dict, axis=1, inplace=True)
    excl_cleaned = pd.concat([excl_cleaned, df])
    
excl_cleaned["StartDate"] = excl_cleaned.StartDate.astype("datetime64[ns]")
excl_cleaned["Sessions"] = excl_cleaned.Sessions.fillna(np.nan).astype("float")

In [24]:
excl_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86552 entries, 0 to 41743
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   person_id                86552 non-null  Int64         
 1   AcademicYear             86552 non-null  object        
 2   NumberOfEnrolments       86175 non-null  Int64         
 3   TotalFixedExclusions     86175 non-null  Int64         
 4   TotalFixedSessions       86175 non-null  Int64         
 5   PermanentExclusionCount  86552 non-null  Int64         
 6   StartDate                86552 non-null  datetime64[ns]
 7   Sessions                 85155 non-null  float64       
 8   Category                 86175 non-null  object        
 9   Term                     86175 non-null  object        
 10  Reason                   86160 non-null  object        
dtypes: Int64(5), datetime64[ns](1), float64(1), object(4)
memory usage: 8.3+ MB


We're left with a 13 column table, in which each entry represents one exclusion. In this format, it now makes less sense for each entry to also have a record of the aggregate exclusion stats for that academic year (i.e. TotalFixedExclusions, TotalFixedSessions, PermanentExclusionCount): that information can already be calculated by aggregating the records in the table, and it adds a layer of confusion to have academic year aggregate stats in each record.

First, I'll do some sanity checks that aggregates match individual instances. Fixed exclusions seem to agree with the aggregate academic year figures:

In [25]:
agg_excl = (excl_cleaned
 .groupby(["person_id", "AcademicYear", "Category"])
 .agg({"Term":"count", 
       "TotalFixedExclusions": "mean",
       "PermanentExclusionCount": "mean",
       "Sessions": "sum",
       "TotalFixedSessions": "mean",
       "NumberOfEnrolments": "mean"})
 .reset_index()
)

agg_fixed_excl = agg_excl[agg_excl.Category == "FIXD"]
exclusions_not_right = agg_fixed_excl.Term != agg_fixed_excl.TotalFixedExclusions
sessions_not_right = agg_fixed_excl.Sessions != agg_fixed_excl.TotalFixedSessions
if len(agg_fixed_excl[exclusions_not_right | sessions_not_right]) == 0:
    print("Fixed Exclusion ocurrences and aggregate column agree")
else:
    print("Fixed Exclusion ocurrences and aggregate column don't agree")

Fixed Exclusion ocurrences and aggregate column agree


3 entries can be found where the permenent exclusion instances don't agree with the aggregate figures:

In [26]:
agg_perm_excl = agg_excl[agg_excl.Category == "PERM"]
exclusions_not_right = agg_perm_excl.Term != agg_perm_excl.PermanentExclusionCount
agg_perm_excl[exclusions_not_right]

,person_id,AcademicYear,Category,Term,TotalFixedExclusions,PermanentExclusionCount,Sessions,TotalFixedSessions,NumberOfEnrolments
17942,12692472,2005/2006,PERM,2,3.0,1.0,0.0,28.0,2.0
29343,13114955,2005/2006,PERM,2,0.0,1.0,0.0,0.0,1.0
41874,13595311,2005/2006,PERM,2,4.0,1.0,0.0,46.0,1.0


As there are only 3 we can inspect them manually. For two, the aggregate just seems to have missed a permanent exclusion - no issues there, as we aren't interested in keeping the aggregate columns anyway:

In [27]:
pd.set_option("display.max_columns", None)
raw_exclusions[raw_exclusions.person_id == 12692472]

,person_id,Sessions30,Term29,Sessions28,Category28,StartDate29,Sessions27,StartDate27,Term27,Reason27,Sessions26,Reason26,Category26,Sessions25,Category25,StartDate24,Reason24,Category24,Sessions23,StartDate23,Term23,Reason23,Category23,StartDate22,Term22,Reason22,Category22,Sessions21,Term21,Reason21,Term20,Reason20,Category20,Sessions19,StartDate19,Term19,StartDate1,StartDate18,Reason18,Sessions17,Sessions6,Sessions10,Sessions3,Sessions12,Reason17,Reason11,Term17,Category17,Category16,Term15,StartDate16,StartDate14,Reason30,Sessions13,Category19,Term13,Reason13,Category13,Category15,Category21,StartDate12,Category12,Term30,Sessions11,StartDate11,Term11,Category2,Category11,StartDate17,Sessions29,Term10,Reason10,Term8,Category10,Term26,StartDate9,Reason9,Reason28,Sessions9,Category9,StartDate28,Sessions8,Sessions18,StartDate8,Reason25,Term9,Category8,StartDate7,Reason7,TotalFixedExclusions,Category18,StartDate30,Sessions24,Term6,Reason16,Sessions7,StartDate15,Reason6,Reason5,Sessions14,Category5,StartDate5,SourceTable,Term5,Sessions20,StartDate6,PupilMatchingRefAnonymous,Term25,Sessions4,StartDate26,StartDate4,Category6,Sessions15,StartDate21,Term4,Term18,Sessions16,StartDate20,StartDate2,Term28,EDRN,Category30,Sessions1,Category27,Reason3,Term3,Category14,Reason2,Term16,Category3,Reason4,PermanentExclusionCount,StartDate3,Reason15,Term14,Sessions22,AcademicYear,Category29,Category1,StartDate13,StartDate10,Category4,Term7,Reason8,Reason14,Reason12,NumberOfEnrolments,StartDate25,Category7,Term1,Term24,Sessions5,Reason19,Reason1,Reason29,TotalFixedSessions,Term12,Term2,Sessions2
3239,12692472,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,NaT,None,None,None,NaT,None,None,None,<NA>,None,None,None,None,None,<NA>,NaT,None,2005-10-04,NaT,None,<NA>,<NA>,<NA>,12,<NA>,None,None,None,None,None,None,NaT,NaT,None,<NA>,None,None,None,None,None,None,NaT,None,None,<NA>,NaT,None,FIXD,None,NaT,None,None,None,None,None,None,NaT,None,None,<NA>,None,None,<NA>,<NA>,NaT,None,None,None,NaT,None,3,None,None,None,None,None,<NA>,NaT,None,DA,<NA>,PERM,2006-06-16,EXC06,Summer,<NA>,NaT,CCF856C035DAB4FEC8,None,10,None,2006-05-15,None,<NA>,NaT,Summer,None,<NA>,NaT,2006-03-28,None,BB823A19E72D30F26F394218D49DD892,None,<NA>,None,OT,Spring,None,OT,None,FIXD,VA,1,2006-04-06,None,None,<NA>,2005/2006,None,PERM,NaT,NaT,FIXD,None,None,None,None,2,None,None,Autumn,None,<NA>,None,DA,None,28,None,Spring,6


In [28]:
raw_exclusions[raw_exclusions.person_id == 541866]

,person_id,Sessions30,Term29,Sessions28,Category28,StartDate29,Sessions27,StartDate27,Term27,Reason27,Sessions26,Reason26,Category26,Sessions25,Category25,StartDate24,Reason24,Category24,Sessions23,StartDate23,Term23,Reason23,Category23,StartDate22,Term22,Reason22,Category22,Sessions21,Term21,Reason21,Term20,Reason20,Category20,Sessions19,StartDate19,Term19,StartDate1,StartDate18,Reason18,Sessions17,Sessions6,Sessions10,Sessions3,Sessions12,Reason17,Reason11,Term17,Category17,Category16,Term15,StartDate16,StartDate14,Reason30,Sessions13,Category19,Term13,Reason13,Category13,Category15,Category21,StartDate12,Category12,Term30,Sessions11,StartDate11,Term11,Category2,Category11,StartDate17,Sessions29,Term10,Reason10,Term8,Category10,Term26,StartDate9,Reason9,Reason28,Sessions9,Category9,StartDate28,Sessions8,Sessions18,StartDate8,Reason25,Term9,Category8,StartDate7,Reason7,TotalFixedExclusions,Category18,StartDate30,Sessions24,Term6,Reason16,Sessions7,StartDate15,Reason6,Reason5,Sessions14,Category5,StartDate5,SourceTable,Term5,Sessions20,StartDate6,PupilMatchingRefAnonymous,Term25,Sessions4,StartDate26,StartDate4,Category6,Sessions15,StartDate21,Term4,Term18,Sessions16,StartDate20,StartDate2,Term28,EDRN,Category30,Sessions1,Category27,Reason3,Term3,Category14,Reason2,Term16,Category3,Reason4,PermanentExclusionCount,StartDate3,Reason15,Term14,Sessions22,AcademicYear,Category29,Category1,StartDate13,StartDate10,Category4,Term7,Reason8,Reason14,Reason12,NumberOfEnrolments,StartDate25,Category7,Term1,Term24,Sessions5,Reason19,Reason1,Reason29,TotalFixedSessions,Term12,Term2,Sessions2


The last seems to have a duplicated record i.e. 2 permanent exclusions taking place on the same date, clearly this has been duplicated in error:

In [29]:
raw_exclusions[raw_exclusions.person_id == 267490]

,person_id,Sessions30,Term29,Sessions28,Category28,StartDate29,Sessions27,StartDate27,Term27,Reason27,Sessions26,Reason26,Category26,Sessions25,Category25,StartDate24,Reason24,Category24,Sessions23,StartDate23,Term23,Reason23,Category23,StartDate22,Term22,Reason22,Category22,Sessions21,Term21,Reason21,Term20,Reason20,Category20,Sessions19,StartDate19,Term19,StartDate1,StartDate18,Reason18,Sessions17,Sessions6,Sessions10,Sessions3,Sessions12,Reason17,Reason11,Term17,Category17,Category16,Term15,StartDate16,StartDate14,Reason30,Sessions13,Category19,Term13,Reason13,Category13,Category15,Category21,StartDate12,Category12,Term30,Sessions11,StartDate11,Term11,Category2,Category11,StartDate17,Sessions29,Term10,Reason10,Term8,Category10,Term26,StartDate9,Reason9,Reason28,Sessions9,Category9,StartDate28,Sessions8,Sessions18,StartDate8,Reason25,Term9,Category8,StartDate7,Reason7,TotalFixedExclusions,Category18,StartDate30,Sessions24,Term6,Reason16,Sessions7,StartDate15,Reason6,Reason5,Sessions14,Category5,StartDate5,SourceTable,Term5,Sessions20,StartDate6,PupilMatchingRefAnonymous,Term25,Sessions4,StartDate26,StartDate4,Category6,Sessions15,StartDate21,Term4,Term18,Sessions16,StartDate20,StartDate2,Term28,EDRN,Category30,Sessions1,Category27,Reason3,Term3,Category14,Reason2,Term16,Category3,Reason4,PermanentExclusionCount,StartDate3,Reason15,Term14,Sessions22,AcademicYear,Category29,Category1,StartDate13,StartDate10,Category4,Term7,Reason8,Reason14,Reason12,NumberOfEnrolments,StartDate25,Category7,Term1,Term24,Sessions5,Reason19,Reason1,Reason29,TotalFixedSessions,Term12,Term2,Sessions2


this raises the issue of duplicated recordings of any sessions - lets take a look for duplicated records of fixed exclusions:

There are 252 such records

In [30]:
pd.set_option("display.max_rows", 300)
dup_cols = ["person_id", "AcademicYear", "StartDate", "Category"]
duplicates = excl_cleaned.duplicated(dup_cols, keep=False)
excl_cleaned[duplicates].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 388 to 31794
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   person_id                252 non-null    Int64         
 1   AcademicYear             252 non-null    object        
 2   NumberOfEnrolments       252 non-null    Int64         
 3   TotalFixedExclusions     252 non-null    Int64         
 4   TotalFixedSessions       252 non-null    Int64         
 5   PermanentExclusionCount  252 non-null    Int64         
 6   StartDate                252 non-null    datetime64[ns]
 7   Sessions                 250 non-null    float64       
 8   Category                 252 non-null    object        
 9   Term                     252 non-null    object        
 10  Reason                   250 non-null    object        
dtypes: Int64(5), datetime64[ns](1), float64(1), object(4)
memory usage: 24.9+ KB


for example, this entry has 23 fixed exclusions, 16 of which are entries that have been duplicated twice:

In [31]:
raw_exclusions.loc[31794:31794]

,person_id,Sessions30,Term29,Sessions28,Category28,StartDate29,Sessions27,StartDate27,Term27,Reason27,Sessions26,Reason26,Category26,Sessions25,Category25,StartDate24,Reason24,Category24,Sessions23,StartDate23,Term23,Reason23,Category23,StartDate22,Term22,Reason22,Category22,Sessions21,Term21,Reason21,Term20,Reason20,Category20,Sessions19,StartDate19,Term19,StartDate1,StartDate18,Reason18,Sessions17,Sessions6,Sessions10,Sessions3,Sessions12,Reason17,Reason11,Term17,Category17,Category16,Term15,StartDate16,StartDate14,Reason30,Sessions13,Category19,Term13,Reason13,Category13,Category15,Category21,StartDate12,Category12,Term30,Sessions11,StartDate11,Term11,Category2,Category11,StartDate17,Sessions29,Term10,Reason10,Term8,Category10,Term26,StartDate9,Reason9,Reason28,Sessions9,Category9,StartDate28,Sessions8,Sessions18,StartDate8,Reason25,Term9,Category8,StartDate7,Reason7,TotalFixedExclusions,Category18,StartDate30,Sessions24,Term6,Reason16,Sessions7,StartDate15,Reason6,Reason5,Sessions14,Category5,StartDate5,SourceTable,Term5,Sessions20,StartDate6,PupilMatchingRefAnonymous,Term25,Sessions4,StartDate26,StartDate4,Category6,Sessions15,StartDate21,Term4,Term18,Sessions16,StartDate20,StartDate2,Term28,EDRN,Category30,Sessions1,Category27,Reason3,Term3,Category14,Reason2,Term16,Category3,Reason4,PermanentExclusionCount,StartDate3,Reason15,Term14,Sessions22,AcademicYear,Category29,Category1,StartDate13,StartDate10,Category4,Term7,Reason8,Reason14,Reason12,NumberOfEnrolments,StartDate25,Category7,Term1,Term24,Sessions5,Reason19,Reason1,Reason29,TotalFixedSessions,Term12,Term2,Sessions2
31794,13577843,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,4,2015-03-17,Spring,DM,FIXD,2015-03-12,Spring,DB,FIXD,2,Spring,PA,Spring,DB,FIXD,4,2015-02-03,Spring,2014-09-04,2015-01-29,DM,2,2,2,2,3,DA,PA,Spring,FIXD,FIXD,Autumn,2014-12-05,2014-11-24,None,4,FIXD,Autumn,PP,FIXD,FIXD,FIXD,2014-11-17,FIXD,None,3,2014-11-17,Autumn,FIXD,FIXD,2015-01-16,None,Autumn,VA,Autumn,FIXD,None,2014-10-08,VA,None,2,FIXD,None,2,2,2014-10-03,None,Autumn,FIXD,2014-10-03,DB,23,FIXD,None,None,Autumn,DM,2,2014-12-05,PP,PP,4,FIXD,2014-09-19,EXC15,Autumn,1,2014-09-19,CCF951C130DEB4FDCB,None,2,None,2014-09-08,FIXD,4,2015-03-05,Autumn,Spring,4,2015-03-03,2014-09-04,None,3C3D23CC077D6BC62E8A465A98CB276,None,1,None,PA,Autumn,FIXD,PA,Autumn,FIXD,PA,0,2014-09-08,DM,Autumn,1,2014/2015,None,FIXD,2014-11-24,2014-10-08,FIXD,Autumn,DB,PP,PA,1,None,FIXD,Autumn,None,2,VP,PA,None,56,Autumn,Autumn,1


10 of these duplicate records are fixed exclusions where the number of sessions excluded disagrees - We need to decide which of the duplicates contains the correct data. 

In [32]:
dup_cols_1 = ["person_id", "AcademicYear", "StartDate", "Category"]
duplicates_1 = excl_cleaned.duplicated(dup_cols_1, keep=False)
dup_cols_2 = ["person_id", "AcademicYear", "StartDate", "Category", "Sessions"]
duplicates_2 = excl_cleaned.duplicated(dup_cols_2, keep=False)
excl_cleaned[duplicates_1 & ~duplicates_2].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 17444 to 41734
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   person_id                10 non-null     Int64         
 1   AcademicYear             10 non-null     object        
 2   NumberOfEnrolments       10 non-null     Int64         
 3   TotalFixedExclusions     10 non-null     Int64         
 4   TotalFixedSessions       10 non-null     Int64         
 5   PermanentExclusionCount  10 non-null     Int64         
 6   StartDate                10 non-null     datetime64[ns]
 7   Sessions                 10 non-null     float64       
 8   Category                 10 non-null     object        
 9   Term                     10 non-null     object        
 10  Reason                   10 non-null     object        
dtypes: Int64(5), datetime64[ns](1), float64(1), object(4)
memory usage: 1010.0+ bytes


Seems logical to assume that an exclusion being extended might result in a duplicate record, so assume the longer of the two exclusions is correct

In [33]:
excl_cleaned = excl_cleaned.sort_values(["person_id", "AcademicYear", "StartDate", "Sessions"])
dup_cols = ["person_id", "AcademicYear", "StartDate", "Category"]
drop_rows = excl_cleaned.duplicated(dup_cols, keep="last")
excl_cleaned = excl_cleaned[~drop_rows].reset_index(drop=True)

There are also a number of entries where the category of exclusion is NA:

In [34]:
na_cats = excl_cleaned.Category.isna()
excl_cleaned[na_cats]

,person_id,AcademicYear,NumberOfEnrolments,TotalFixedExclusions,TotalFixedSessions,PermanentExclusionCount,StartDate,Sessions,Category,Term,Reason
639,47210,2002/2003,<NA>,<NA>,<NA>,1,2002-06-20,NaN,None,None,None
1002,69608,2005/2006,<NA>,<NA>,<NA>,1,2005-02-22,NaN,None,None,None
1095,74244,2005/2006,<NA>,<NA>,<NA>,1,2004-09-13,NaN,None,None,None
1313,86757,2005/2006,<NA>,<NA>,<NA>,1,2005-05-04,NaN,None,None,None
1319,87056,2005/2006,<NA>,<NA>,<NA>,1,2004-11-22,NaN,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
85765,13750097,2005/2006,<NA>,<NA>,<NA>,1,2004-09-13,NaN,None,None,None
85903,13752696,2002/2003,<NA>,<NA>,<NA>,1,2002-03-21,NaN,None,None,None
85926,13753528,2004/2005,<NA>,<NA>,<NA>,1,2004-05-13,NaN,None,None,None
86316,13761292,2004/2005,<NA>,<NA>,<NA>,1,2004-07-21,NaN,None,None,None


The permanent exclusion count for all of these entries is 1. We know from DfE feedback, these are pre-2006 records where the reporting requirements were different. We can safely update the "Category" for all these records to "PERM":

In [35]:
excl_cleaned.loc[na_cats, "Category"] = "PERM"

We can now drop the aggregate columns as they don't add any extra info that can't be derived from the data itself:

In [36]:
agg_cols = ["TotalFixedExclusions", "TotalFixedSessions", "PermanentExclusionCount"]
excl_cleaned.drop(agg_cols, axis=1, inplace=True)

I also think the NumberOfEnrolments figure should be dropped. The DfE dictionary describes this variable as:

*Pupil appears more than once in this table with exclusions from another school - 1 = Has exclusions from one school in reporting period, 2 = Appears twice in the table with exclusions from two schools, 3 = Appears three times in the table with exclusions from three schools etc.*

This doesn't make sense when you check the data. 

The "appears more than once" presumably means a person_id appears more than once for a given academic year, as this data is collated from multiple tables recorded for each academic year. Checking this, it is clear that there are a number of entries where individuals that appear multiple times aren't recorded in this figure:

In [37]:
two_enrols = excl_cleaned.NumberOfEnrolments == 2
two_enrol_ids = excl_cleaned[two_enrols].person_id.unique()
two_entries_df = (raw_exclusions
                .sort_values("person_id") 
                .groupby(["person_id", "AcademicYear"])[["EDRN"]] 
                .count() 
                .rename({"EDRN": "n_entries"}, axis=1)
                .reset_index())
two_entries_df = two_entries_df[two_entries_df.n_entries == 2]
two_entries_ids = two_entries_df.person_id.unique()
n_not_recorded = len([person_id for person_id in two_entries_ids
                  if person_id not in two_enrol_ids])
n_not_recorded

15

Also, it isn't really necessary to know the number of different schools a student is enrolled at within an academic year, based on the way the data is now formatted. We have individual dated records for each exclusion, so the fact they might have occurred at different schools isn't much use, as we don't have any identifiers for the school). Given that the observations are misleading (didn't pass QA) and they don't add any useful information, I believe they should be dropped.

In [38]:
excl_cleaned.drop("NumberOfEnrolments", axis=1, inplace=True)

In [39]:
excl_cleaned.to_gbq("yhcr-prd-phm-bia-core.CY_FDM_DepartmentForEducation.exclusions_cleaned")

100%|██████████| 1/1 [00:00<00:00, 9383.23it/s]


In [20]:
excl_cleaned

,index,person_id,AcademicYear,StartDate,Sessions,Category,Term,Reason
0,707,289,2005/2006,2006-03-15,4.0,FIXD,Spring,VA
1,5776,289,2006/2007,2006-10-03,4.0,FIXD,Autumn,OT
2,21429,1116,2011/2012,2012-07-12,6.0,FIXD,Summer,BU
3,4774,1127,2006/2007,2007-03-14,4.0,FIXD,Spring,VP
4,24144,1149,2012/2013,2012-10-08,4.0,FIXD,Autumn,SM
...,...,...,...,...,...,...,...,...
86417,37639,13763893,2016/2017,2016-11-03,10.0,FIXD,Autumn,DB
86418,37639,13763893,2016/2017,2017-02-09,20.0,FIXD,Spring,DB
86419,37639,13763893,2016/2017,2017-03-23,24.0,FIXD,Spring,DB
86420,6391,13763932,2006/2007,2006-09-28,10.0,FIXD,Autumn,OT
